# Figure 4 (PE-Aged) Full Analysis Notebook

This notebook performs grid construction and CCI analyses, then generates panels 4E–4H for the PE-Aged sample. Data are loaded directly from `./data/`. Results are saved to `./results/figure4_PEAged/`.

## 0. Setup & Imports

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import math
import numpy as np
import pandas as pd
import scanpy as sc
import squidpy as sq
import stlearn as st
import matplotlib.pyplot as plt

import cv2
from pathlib import Path
from skimage.morphology import remove_small_objects, remove_small_holes, opening, closing, disk
from scipy.ndimage import distance_transform_edt
from spatialdata_io.readers.xenium import xenium, xenium_aligned_image

# Directories
DATA_DIR = Path("./data")
RESULTS_DIR = Path("./results/figure4_PEAged")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Ensure numpy compatibility
import numpy as _np; _np.int = int


## 1. Load Xenium Data & AnnData

In [ ]:
# Load SpatialData for PE-Aged
sdata = xenium(
    image=DATA_DIR/"morphology_PE-Aged.ome.tif",
    matrix=DATA_DIR/"transcripts_PE-Aged.parquet",
    cell_boundaries=DATA_DIR/"cell_boundaries_PE-Aged.parquet",
    cells=DATA_DIR/"cells_PE-Aged.parquet",
    nucleus_boundaries=DATA_DIR/"nucleus_boundaries_PE-Aged.parquet"
)
he_img = xenium_aligned_image(
    DATA_DIR/"morphology_PE-Aged.ome.tif",
    DATA_DIR/"transcripts_PE-Aged.parquet"
)
sdata.images["he_image_aligned"] = he_img

# Load AnnData (with cell_type2, cell_niche)
adata = sc.read_h5ad(DATA_DIR/"PE-Aged.h5ad")


## 2. Grid Construction for CCI Analysis

In [ ]:
# Build grid for cell-cell interaction (CCI)
n_grid = 250
grid = st.tl.cci.grid(adata, n_row=n_grid, n_col=n_grid, use_label="cell_type2")

# Copy spatial metadata and set quality
grid.uns['spatial'] = {"library_id": adata.uns['spatial']['library_id']}
adata.uns['spatial']['library_id']['use_quality'] = 'hires'
grid.uns['spatial']['library_id']['use_quality'] = 'hires'


## 3. Ligand–Receptor & CCI Computations

In [ ]:
# Load LR pairs for mouse
lrs = st.tl.cci.load_lrs(['connectomeDB2020_lit'], species='mouse')
print(f"Loaded {len(lrs)} LR pairs")

# Compute neighborhood-based CCI scores
st.tl.cci.run(
    grid, lrs,
    min_spots=20,
    distance=50,
    n_pairs=1000,
    n_cpus=4
)

# Permutation test for significant interactions
st.tl.cci.run_cci(
    grid,
    use_label='cell_type2',
    min_spots=2,
    spot_mixtures=True,
    cell_prop_cutoff=0.1,
    sig_spots=True,
    n_perms=500,
    n_cpus=None
)


## 4. Figure 4E & 4F: Fibrotic Niche CCI & Top LR Pairs (PE-Aged)

In [ ]:
# Panel 4E: Fibrotic niche CCI heatmap
if 'lr_sig_scores' in grid.obsm:
    fib_mask = grid.obs['cell_niche']=='Fibrotic Niche'
    st.pl.cci.heatmap(
        grid[fib_mask,:],
        use_label='cell_type2',
        figsize=(8,6),
        show=False
    )
    plt.title("Figure 4E: PE-Aged Fibrotic Niche CCI Heatmap")
    plt.savefig(RESULTS_DIR/"Figure4E_PEAged_fibrotic_cci.png", dpi=300, bbox_inches='tight')
    plt.close()

# Panel 4F: Top 10 LR pairs in fibrotic niche
if 'lr_sig_scores' in grid.obsm:
    scores = grid.obsm['lr_sig_scores']
    names  = grid.uns.get('lr_pair_names')
    fib_mask = grid.obs['cell_niche']=='Fibrotic Niche'
    summed = scores[fib_mask].sum(axis=0)
    idx = np.argsort(summed)[-10:][::-1]
    df = pd.DataFrame({
        'lr_pair': [names[i] for i in idx] if names else idx,
        'score': summed[idx]
    })
    ax = df.plot.barh(x='lr_pair', y='score', legend=False, figsize=(6,6))
    ax.invert_yaxis()
    plt.title("Figure 4F: PE-Aged Top 10 LR Pairs (Fibrotic)")
    plt.xlabel("Summed Score")
    plt.tight_layout()
    plt.savefig(RESULTS_DIR/"Figure4F_PEAged_fibrotic_top10.png", dpi=300, bbox_inches='tight')
    plt.close()


## 5. Figure 4G & 4H: Remote Niche CCI & Top LR Pairs (PE-Aged)

In [ ]:
# Panel 4G: Remote niche CCI heatmap
if 'lr_sig_scores' in grid.obsm:
    rem_mask = grid.obs['cell_niche']=='Remote'
    st.pl.cci.heatmap(
        grid[rem_mask,:],
        use_label='cell_type2',
        figsize=(8,6),
        show=False
    )
    plt.title("Figure 4G: PE-Aged Remote Niche CCI Heatmap")
    plt.savefig(RESULTS_DIR/"Figure4G_PEAged_remote_cci.png", dpi=300, bbox_inches='tight')
    plt.close()

# Panel 4H: Top 10 LR pairs in remote niche
if 'lr_sig_scores' in grid.obsm:
    scores = grid.obsm['lr_sig_scores']
    names  = grid.uns.get('lr_pair_names')
    rem_mask = grid.obs['cell_niche']=='Remote'
    summed = scores[rem_mask].sum(axis=0)
    idx = np.argsort(summed)[-10:][::-1]
    df = pd.DataFrame({
        'lr_pair': [names[i] for i in idx] if names else idx,
        'score': summed[idx]
    })
    ax = df.plot.barh(x='lr_pair', y='score', legend=False, figsize=(6,6))
    ax.invert_yaxis()
    plt.title("Figure 4H: PE-Aged Top 10 LR Pairs (Remote)")
    plt.xlabel("Summed Score")
    plt.tight_layout()
    plt.savefig(RESULTS_DIR/"Figure4H_PEAged_remote_top10.png", dpi=300, bbox_inches='tight')
    plt.close()
